# visualize video


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
def save_episode_video(episode_data, key, output_dir):
    """Save the episode video to a file."""
    frames = []

    # Collect frames from episode data
    rgb_data = episode_data['episode_data']['camera_info']['pano_camera_0']['rgb']
    for frame in rgb_data:
        # Convert the frame to a PIL image and then to a NumPy array
        pil_image = Image.fromarray(frame)
        frames.append(np.array(pil_image))

    # Define output video file path
    output_file = os.path.join(output_dir, f"episode_video_{key}.mp4")

    # Check the dimensions of the first frame
    if len(frames) > 0:
        height, width, layers = frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_file, fourcc, 6, (width, height))

        # Write frames to video
        for frame in frames:
            video_writer.write(frame)

        # Release the video writer
        video_writer.release()
        print(f"Video saved successfully to {output_file}")
    else:
        print("No frames to save to video.")



In [8]:
import gzip
import json
import copy
import os
starrt_pos_dict = {}
episode_id = []
trajectory_id_list = []

target_save_file = 'vlmaps/docker/valid_paths/sub_success_paths_room_1212.txt'
target_save_file_id = 'vlmaps/docker/valid_paths/sub_success_paths_id_1212.txt' 
with open(target_save_file, 'w') as f:
    pass
with open(target_save_file_id, 'w') as f:
    pass
def get_sub_trajectory_id_list():
    with open('vlmaps/docker/valid_paths/success_paths_id_all.txt', 'r') as f:
        sub_trajectory_id_list = [int(line.strip()) for line in f.readlines()]
        # 重新按大小排序
        sub_trajectory_id_list.sort()
    return sub_trajectory_id_list


def find_gpu_id(trajectory_id,num_of_gpus,trajectory_id_list):
    for idx, id in enumerate(trajectory_id_list):
        if (id == trajectory_id):
            return idx % num_of_gpus
def judge_stair_exists(path):
    '''
    path_z: ndarray, if max(path_z)- min(path_z) > 1.5, then there is a stair
    '''
    path_z = []
    for path in path:
        path_z.append(path[1])
    return max(path_z)- min(path_z) > 1.5

def load_data(split,num_of_gpus):
    ''' Load data based on VLN-CE
    '''
    # dataset_root_dir = base_data_dir # '../VLN/VLNCE/R2R_VLNCE_v1-3'
    dataset_root_dir = '/ssd/xiaxinyuan/code/VLN/VLNCE/R2R_VLNCE_v1-3'
    scene_id_list = []
    total_scans = []
    load_data = []
    split_data = [[] for _ in range(num_of_gpus)]
    trajectory_id_list = get_sub_trajectory_id_list()
    print(len(trajectory_id_list))
    for split in splits:
            with gzip.open(os.path.join(dataset_root_dir, f"{split}", f"{split}.json.gz"), 'rt', encoding='utf-8') as f:
                data = json.load(f)
            for item in data["episodes"]:
                scene_id = item['scene_id'].split('/')[1]
                if scene_id not in scene_id_list:
                    scene_id_list.append(scene_id)
            for item in data["episodes"]:
                if not judge_stair_exists(item['reference_path'])  and (item['trajectory_id'] in trajectory_id_list):
                    instruction = item['instruction']['instruction_text']
                    if not ('room' in instruction or 'hall' in instruction or 'way' in instruction):
                        continue
                    scene_id = item['scene_id'].split('/')[1]
                    print(scene_id,item['trajectory_id'],item['episode_id'],item['instruction']['instruction_text'])
                    with open(target_save_file, 'a') as f:
                        f.write(f"{scene_id},{item['trajectory_id']},{item['episode_id']},{item['instruction']['instruction_text']}\n")
                    with open(target_save_file_id, 'a') as f:
                        f.write(f"{item['episode_id']}\n")
                    # gpu_id = find_gpu_id(scene_id, num_of_gpus, scene_id_list)
                    # split_data[gpu_id].append((scene_id,item['trajectory_id'],item['episode_id']))
                    

    return split_data
    # return load_data, list(set(total_scans))
# splits = ['train', 'val_seen', 'val_unseen', 'test']
splits = ['train']
# splits = ['val_seen']
# splits = ['val_seen']
num_of_gpus = 4
for split in splits:
    split_data = load_data(split,num_of_gpus)
    print(split_data)
    


715
7y3sRwLe3Va 4 1 Go around the right side of the center unit and stop by the right side doorway with the dining table and mirror in it. 
7y3sRwLe3Va 4 3 Turn to the right, go past the refrigerator.  Turn left and walk to the point where you're to the hallway by the entry and dining room area. 
5LpN3gDmAk7 5 4 Enter the bedroom and go around the bed. Go to the closet on the far wall on the left. Stop in the doorway. 
5LpN3gDmAk7 5 5 Walk around the bed in the bedroom. Walk into the closet area on the other side of the bed that has empty shelves inside. 
5LpN3gDmAk7 5 6 Go into the room, turn left at the foot of the bed, and left again at the far side of the bed. Go to the doorway to the closet on the right and stop. 
7y3sRwLe3Va 6 7 Exit the bedroom and turn right.Walk past the staircase and to the door on the right in front of an entryway. Go through the door and you'll be in an office area. 
7y3sRwLe3Va 6 8 Go through the doorway to the hall.  Go down the hall past teh landing.  Ta

In [10]:
a = [1,2,3]
print(a[4:])

[]


In [1]:
description_txt_file = 'vlmaps/docker/valid_paths/sub_success_paths_room_1212.txt'
target_txt_file = 'vlmaps/docker/valid_paths/sub_success_paths_id_1212.txt'
with open(description_txt_file, 'r') as f:
    for line in f.readlines():
        # if the line can be seperated by space with more than 2 parts, then it is a valid line
        if len(line.split()) > 2:
            # write the line to the target file, seperate by ','
            if  len(line.split()[0])!=11:
                continue
            with open(target_txt_file, 'a') as f:
                f.write(line.split()[2]+'\n')
                print(line.split()[2])


2
28
68
69
90
92
100
102
115
127
128
129
172
217
247
248
249
358
359
407
445
599
600
609
626
733
745
749
840
894
919
926
929
960
962
980
987
988
1048
1050
1071
1073
1078
1082
1085
1100
1135
1163
1215
1239
1242
1243
1257
1263
1264
1338
1364
1366
1390
1391
1441
1442
1444
1468
1541
1543
1802
1839
1840
1864
1910
1916
1923
1930
1947
1997
2010
2011
2038
2138
2145
2172
2173
2200
2209
2210
2237
2343
2354
2356
2410
2428
2441
2442
2443
2461
2507
2558
2595
2627
2628
2629
2682
2708
2709
2717
2735
2736
2737
2776
2819
2960
2962
2964
3009
3034
3044
3100
3173
3228
3308
3310
3338
3339
3340
3367
3377
3378
3379
3413
3414
3435
3436
3507
3508
3546
3573
3599
3600
3618
3635
3636
3637
3678
3685
3798
3806
3807
3808
3839
3882
3885
3891
3892
4022
4037
4039
4075
4169
4170
4248
4289
4348
4394
4445
4447
4451
4466
4467
4468
4524
4525
4604
4605
4606
4676
4677
4732
4806
4850
4851
4861
4871
4872
4873
4967
4998
4999
5006
5007
5008
5013
5014
5037
5039
5040
5041
5085
5086
5118
5122
5124
5131
5137
5140
5213
5215
5219
5220


In [3]:
from vlmaps.application_my.utils import check_valid_parsed_instruction


import pytest

def test_valid_instructions():
    """测试有效的指令"""
    valid_instructions = [
        "self.turn(90)",
        "self.turn(90.5)",
        "self.turn(-180)",
        "self.move_forward(1.5)",
        "self.move_to_object('chair')",
        'self.move_to_object("table")',
        "self.move_to_left('wall')",
        "self.move_to_right('door')",
        "self.move_in_between('desk', 'chair')",
        "self.face('window')",
        "self.turn_absolute(45.5)",
        "self.get_pos('sofa')",
        "self.get_contour('bed')",
        "self.with_object_on_left('cabinet')",
        "self.with_object_on_right('shelf')"
    ]
    
    for instruction in valid_instructions:
        assert check_valid_parsed_instruction(instruction) == True

def test_invalid_instructions():
    """测试无效的指令"""
    invalid_instructions = [
        # 无效的函数名
        "self.invalid_function(90)",
        "self.unknown_action('chair')",
        
        # 参数数量错误
        "self.turn()",
        "self.turn(90, 180)",
        "self.move_to_object()",
        "self.move_in_between('desk')",
        "self.move_in_between('desk', 'chair', 'table')",
        
        # 参数类型错误
        "self.turn('90')",  # 数字不应该用引号
        "self.turn(chair)", # 缺少引号的字符串
        "self.move_forward('fast')", # 应该是数字
        "self.move_to_object(123)", # 应该是字符串
        
        # 语法错误
        "self.turn(90",  # 缺少右括号
        "self.move_to_object'chair')", # 缺少左括号
        "turn(90)", # 缺少self
        
        # 引号不匹配
        "self.move_to_object('chair\")",
        "self.move_in_between('desk\", \"chair')",
    ]
    
    for instruction in invalid_instructions:
        try:
            check_valid_parsed_instruction(instruction)
        except Exception as e:
            print(e)

def test_edge_cases():
    """测试边界情况"""
    edge_cases = [
        # 特殊字符
        "self.move_to_object('chair-1')",
        "self.move_to_object('dining_table')",
        "self.move_to_object('kitchen/living room')",
        
        # 空格处理
        "self.turn( 90 )",
        "self.move_to_object( 'chair' )",
        "self.move_in_between( 'desk' , 'chair' )",
        
        # 小数点和负数
        "self.turn(-90.5)",
        "self.move_forward(.5)",
        "self.turn_absolute(-0.0)",
    ]
    
    for instruction in edge_cases:
        assert check_valid_parsed_instruction(instruction) == True

def test_parameter_combinations():
    """测试参数组合"""
    # 生成不同的数字组合
    numbers = [0, 1, -1, 0.5, -0.5, 90, 180, 360, -360]
    for num in numbers:
        assert check_valid_parsed_instruction(f"self.turn({num})") == True
        assert check_valid_parsed_instruction(f"self.move_forward({num})") == True
        assert check_valid_parsed_instruction(f"self.turn_absolute({num})") == True
    
    # 生成不同的字符串组合
    objects = ['chair', 'table', 'desk', 'sofa', 'bed', 'door', 'window']
    for obj in objects:
        assert check_valid_parsed_instruction(f"self.move_to_object('{obj}')") == True
        assert check_valid_parsed_instruction(f"self.face('{obj}')") == True
    
    # 测试两个参数的组合
    for obj1 in objects[:3]:
        for obj2 in objects[3:]:
            assert check_valid_parsed_instruction(f"self.move_in_between('{obj1}', '{obj2}')") == True


test_valid_instructions()
test_invalid_instructions()
test_edge_cases()
test_parameter_combinations()
print("所有测试通过！")

指令解析错误: 无效的函数名: invalid_function
指令解析错误: 无效的函数名: unknown_action
指令解析错误: 函数 turn 需要 1 个参数，但收到了 0 个
指令解析错误: 函数 turn 需要 1 个参数，但收到了 2 个
指令解析错误: 函数 move_to_object 需要 1 个参数，但收到了 0 个
指令解析错误: 函数 move_in_between 需要 2 个参数，但收到了 1 个
指令解析错误: 函数 move_in_between 需要 2 个参数，但收到了 3 个
指令解析错误: 参数 chair 应该是数字类型
指令解析错误: 参数 fast 应该是数字类型
指令解析错误: 参数 123 应该是字符串类型，不应该是数字
指令解析错误: list index out of range
指令解析错误: list index out of range
所有测试通过！


In [4]:
test_action_lists = ["self.move_to_object(1)", "self.move_to_object('1')", "self.move_to_object(1.0)", "self.move_to_object('1.0')"]
for action in test_action_lists:
    try:
        check_valid_parsed_instruction(action)
        print(f"valid action: {action}")
    except Exception as e:
        print(e)

指令解析错误: 函数 move_to_object 需要 1 个参数，但收到了 0 个
valid action: self.move_to_object('1')
指令解析错误: 函数 move_to_object 需要 1 个参数，但收到了 0 个
valid action: self.move_to_object('1.0')
